In [1]:
import os
import time
import wandb
from tqdm.notebook import tqdm
from copy import deepcopy

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

from utils import seed_everything
from data import ImageDataset, stratified_kfold, get_train_transforms, get_valid_transforms
from model import SpecieClassifier
from scheduler import CosineAnnealingWarmupRestarts

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
class Config:
    checkpoint = '/root/dl_whale_classification/pths_specie'
    test = False
    tta = False
    resume_epoch = 0
    iters_to_accumulate = 1
    resume_root = None
    wandb_log = True
    model_name = 'tf_efficientnet_b3_ns'
    fold = 0
    n_split = 5
    seed = 2022
    data_dir = '/root/data2/' # root/data/train_images
    root_dir = '.'
    batch_size = 24
    lr = 1e-4
    weight_decay = 0.0005
    epoch = 20
    exp_name = 'test'
config = Config()

In [3]:
seed_everything(2022)

In [4]:
df = pd.read_csv('/root/data/train.csv')
df.species.replace({"globis": "short_finned_pilot_whale",
                  "pilot_whale": "short_finned_pilot_whale",
                  "kiler_whale": "killer_whale",
                  "bottlenose_dolpin": "bottlenose_dolphin"}, inplace=True)

In [5]:
specie_unique = df.species.unique()
specie_indices = range(len(specie_unique))
species2idx = {k: v for k, v in zip(specie_unique, specie_indices)}
df.species = df.species.map(species2idx)

individual_unique = df.individual_id.unique()
individual_indices = range(len(individual_unique))
individual2idx = {k : v for k, v in zip(individual_unique, individual_indices)}
df.individual_id = df.individual_id.map(individual2idx)

In [6]:
print('num_species', len(df.species.unique()))
print('num_individual', len(df.individual_id.unique()))
      

num_species 26
num_individual 15587


In [7]:
df_single = df[df['individual_id'].map(df['individual_id'].value_counts()) == 1]
df_others = df[df['individual_id'].map(df['individual_id'].value_counts()) > 1]

In [8]:
train_single, valid_single = stratified_kfold(df=df_single, fold=config.fold, n_split=config.n_split, seed=config.seed, target_col='species')
train_others, valid_others = stratified_kfold(df=df_others, fold=config.fold, n_split=config.n_split, seed=config.seed, target_col='species')

/opt/conda/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [9]:
train_single_indices = np.take(df_single.index.to_numpy(), train_single)    
train_others_indices = np.take(df_others.index.to_numpy(), train_others)
valid_single_indices = np.take(df_single.index.to_numpy(), valid_single)
valid_others_indices = np.take(df_others.index.to_numpy(), valid_others)

In [10]:
full_train_indices = np.sort(np.concatenate((train_single_indices, train_others_indices), axis=0))

In [11]:
fnames, labels = df['image'].values, df['species'].values

In [12]:
fnames_train, labels_train = fnames[full_train_indices], labels[full_train_indices]
fnames_valid_single, labels_valid_single = fnames[valid_single_indices], labels[valid_single_indices]
fnames_valid_others, labels_valid_others = fnames[valid_others_indices], labels[valid_others_indices]

In [13]:
train_transforms = get_train_transforms()
valid_transforms = get_valid_transforms()

In [14]:
train_dataset = ImageDataset(path=fnames_train, target=labels_train, transform=train_transforms, root=config.data_dir + '/train_detec_512_v3/')
valid_dataset_single = ImageDataset(path=fnames_valid_single, target=labels_valid_single, transform=valid_transforms, root=config.data_dir + '/train_detec_512_v3/')
valid_dataset_others = ImageDataset(path=fnames_valid_others, target=labels_valid_others, transform=valid_transforms, root=config.data_dir + '/train_detec_512_v3/')

In [15]:
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=8, pin_memory=True, drop_last=True)
valid_loader_single = DataLoader(valid_dataset_single, batch_size=config.batch_size*2, shuffle=False, num_workers=8, pin_memory=True)
valid_loader_others = DataLoader(valid_dataset_others, batch_size=config.batch_size*2, shuffle=False, num_workers=8, pin_memory=True)

In [16]:
model = SpecieClassifier('tf_efficientnet_b3_ns').to(device)

In [17]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

In [18]:
cosine_annealing_scheduler_arg = dict(
    first_cycle_steps=len(train_dataset)//config.batch_size*config.epoch,
    cycle_mult=1.0,
    max_lr=config.lr,
    min_lr=1e-07,
    warmup_steps=len(train_dataset)//config.batch_size*3, # wanrm up 0~3 epoch
    gamma=0.9
)

In [19]:
scheduler = CosineAnnealingWarmupRestarts(optimizer, **cosine_annealing_scheduler_arg)

In [20]:
if config.wandb_log:
    run = wandb.init(config=config.__dict__,
                project=config.model_name, 
                settings=wandb.Settings(start_method="thread"), 
                name=f"{config.exp_name}_fold{config.fold}",
                reinit=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jcdata (use `wandb login --relogin` to force relogin)


In [21]:
grad_scaler = torch.cuda.amp.GradScaler()

best_model = None
best_acc,  best_epoch = 0, 0

In [22]:
if config.resume_root is not None:
    check = torch.load(config.resume_root)
    model.load_state_dict(check['model'])
    optimizer.load_state_dict(check['optimizer'])
    scheduler.load_state_dict(check['scheduler'])
    print('loaded checkpoint')

# Train Functions

In [23]:
def train_one_epoch(model, optimizer, criterion, loader, scheduler, scaler=None, iters_to_accumulate=1):
    model.train()
    
    match = 0
    top_k_match = 0
    
    losses, y_true, y_pred = [], [], []
    for i, (x, y) in enumerate(tqdm(loader)):
        x, y = x.to(device), y.to(device)
        
        if scaler is not None:
            with torch.cuda.amp.autocast():
                output = model(x) 
                loss = criterion(output, y)
                
            scaler.scale(loss).backward()

            if ((i + 1) % iters_to_accumulate == 0) or ((i + 1) == len(loader)):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
        

        else:
            output = model(x, y) 
            loss = criterion(output, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        match += (torch.argmax(output.detach().cpu(), dim=-1) == y.detach().cpu()).sum()
        top_k_match += (output.detach().topk(2, dim=-1).indices.cpu() == y.detach().cpu()[:, None]).sum()
        scheduler.step()

        losses.append(loss.detach().cpu().item())
        # if i == 10:
        #     break
        
    return np.mean(losses), match, top_k_match


In [24]:
def valid_one_epoch(model, criterion, loader, tta=False):
    model.eval()
    
    match = 0
    top_k_match = 0

    losses, y_trues, y_preds, tta_embed = [], [], [], []
    with torch.no_grad():
        for idx, (x, y) in tqdm(enumerate(loader)):
            x, y = x.to(device), y.to(device)
            with torch.cuda.amp.autocast():
                output = model(x) 
                loss = criterion(output, y)
  
            match += (torch.argmax(output.detach().cpu(), dim=-1) == y.detach().cpu()).sum()
            top_k_match += (output.detach().topk(2, dim=-1).indices.cpu() == y.detach().cpu()[:, None]).sum()
            losses.append(loss.detach().cpu().item())
            
            y_preds.extend(np.argmax(output.detach().cpu().numpy(), axis=-1))
            y_trues.extend(y.detach().cpu().numpy())
            
    return np.mean(losses), match, top_k_match, np.array(y_preds), np.array(y_trues)

# Train

In [ ]:
start = time.time()
os.makedirs(f"{config.checkpoint}/{config.model_name}", exist_ok=True)

print('Start Training')
for i in range(config.resume_epoch, config.epoch):
    print(f"epoch: {i}")
    lr = scheduler.get_lr()[0]
    train_loss, train_acc, train_top_k = train_one_epoch(model, optimizer, criterion, train_loader, scheduler, grad_scaler, config.iters_to_accumulate)
    valid_loss1, valid_acc1, valid_top_k1, valid_preds1, valid_trues1 = valid_one_epoch(model, criterion, valid_loader_single, config.tta)
    valid_loss2, valid_acc2, valid_top_k2, valid_preds2, valid_trues2 = valid_one_epoch(model, criterion, valid_loader_others, config.tta)

    print(f"train loss {train_loss :.4f} acc {train_acc :.4f} topk {train_top_k :.4f}")
    print(f"valid loss (single) {valid_loss1 :.4f} acc {valid_acc1/len(valid_loader_single.dataset) :.4f} topk {valid_top_k1/len(valid_loader_single.dataset) :.4f}")
    print(f"valid loss (others) {valid_loss2 :.4f} acc {valid_acc2/len(valid_loader_others.dataset) :.4f} topk {valid_top_k2/len(valid_loader_others.dataset) :.4f}")
    print(f"lr {lr} time {time.time() - start :.2f}s")
    
    print("-"*20, f"class (single)", "-"*20)
    for i in range(26):
        print(f"{i:02d}-#{(valid_trues1==i).sum():04d} : {((valid_preds1==i)&(valid_trues1==i)).sum()/(valid_trues1==i).sum():04f}", end=' / ') 
        if i % 4 == 3 and i > 0 or i == 25:
            print()

    print("-"*20, f"class (others)", "-"*20)
    for i in range(26):
        print(f"{i:02d}-#{(valid_trues2==i).sum():04d} : {((valid_preds2==i)&(valid_trues2==i)).sum()/(valid_trues2==i).sum():04f}", end=' / ') 
        if i % 4 == 3 and i > 0 or i == 25:
            print()
                                     
    if best_acc < valid_acc1/len(valid_loader_single.dataset):
        best_acc = valid_acc1/len(valid_loader_single.dataset)
        best_epoch = i
        print(f'best acc updated {best_acc}')
        best_model_dict = {
            'model': deepcopy(model.state_dict()),
            'optimizer': deepcopy(optimizer.state_dict()),
            'scheduler': deepcopy(scheduler.state_dict()),
        }
        torch.save(best_model_dict, f"{config.checkpoint}/{config.model_name}/best.pt")

    if config.wandb_log:
        wandb_dict = {
            'train loss': train_loss,
            'train acc': train_acc,
            'valid loss (single)': valid_loss1,
            'valid acc (single)': valid_acc1 / len(valid_loader_single.dataset),
            'valid topk (single)': valid_top_k1 / len(valid_loader_single.dataset),
            'valid loss (others)': valid_loss2,
            'valid acc (others)': valid_acc2 / len(valid_loader_others.dataset),
            'valid topk (others)': valid_top_k2 / len(valid_loader_others.dataset),
            'learning rate': scheduler.get_lr()[0],
        }
        wandb.log(wandb_dict)
        
os.rename(f"{config.checkpoint}/{config.model_name}/best.pt", \
          f"{config.checkpoint}/{config.model_name}/fold{config.fold}_epoch{best_epoch}_{best_acc:.04f}.pt")

Start Training
epoch: 0



train loss 1.6808 acc 22959.0000 topk 27200.0000
valid loss (single) 0.5072 acc 0.8553 topk 0.9174
valid loss (others) 0.3431 acc 0.9053 topk 0.9469
lr 1e-07 time 1738.09s
-------------------- class (single) --------------------
00-#0228 : 0.833333 / 01-#0370 : 0.970270 / 02-#0008 : 0.625000 / 03-#0051 : 0.941176 / 
04-#0052 : 1.000000 / 05-#0007 : 0.714286 / 06-#0062 : 0.838710 / 07-#0185 : 0.978378 / 
08-#0011 : 1.000000 / 09-#0061 : 1.000000 / 10-#0002 : 0.000000 / 11-#0031 : 0.870968 / 
12-#0045 : 0.244444 / 13-#0492 : 0.989837 / 14-#0012 : 0.000000 / 15-#0017 : 0.117647 / 
16-#0104 : 0.894231 / 17-#0021 : 0.000000 / 18-#0036 : 0.000000 / 19-#0004 : 0.000000 / 
20-#0010 : 0.000000 / 21-#0027 : 0.000000 / 22-#0007 : 0.000000 / 23-#0005 : 0.000000 / 
24-#0002 : 0.000000 / 25-#0002 : 0.000000 / 
-------------------- class (others) --------------------
00-#0111 : 0.864865 / 01-#1109 : 0.963030 / 02-#0657 : 0.856925 / 03-#2105 : 0.983848 / 
04-#1436 : 0.990251 / 05-#0315 : 0.901587 / 0

<ipython-input-25-8f53dd3d1892>:25: RuntimeWarning: invalid value encountered in long_scalars
  print(f"{i:02d}-#{(valid_trues2==i).sum():04d} : {((valid_preds2==i)&(valid_trues2==i)).sum()/(valid_trues2==i).sum():04f}", end=' / ')


epoch: 1



train loss 0.2762 acc 37601.0000 topk 39286.0000
valid loss (single) 0.1465 acc 0.9552 topk 0.9811
valid loss (others) 0.1165 acc 0.9700 topk 0.9872
lr 3.34e-05 time 3482.14s
-------------------- class (single) --------------------
00-#0228 : 0.991228 / 01-#0370 : 0.994595 / 02-#0008 : 0.500000 / 03-#0051 : 0.882353 / 
04-#0052 : 1.000000 / 05-#0007 : 0.714286 / 06-#0062 : 0.887097 / 07-#0185 : 0.956757 / 
08-#0011 : 1.000000 / 09-#0061 : 1.000000 / 10-#0002 : 1.000000 / 11-#0031 : 0.935484 / 
12-#0045 : 0.844444 / 13-#0492 : 0.989837 / 14-#0012 : 0.666667 / 15-#0017 : 0.941176 / 
16-#0104 : 0.961538 / 17-#0021 : 0.857143 / 18-#0036 : 0.916667 / 19-#0004 : 0.250000 / 
20-#0010 : 0.600000 / 21-#0027 : 0.962963 / 22-#0007 : 0.000000 / 23-#0005 : 0.200000 / 
24-#0002 : 0.000000 / 25-#0002 : 0.000000 / 
-------------------- class (others) --------------------
00-#0111 : 0.981982 / 01-#1109 : 0.982867 / 02-#0657 : 0.957382 / 03-#2105 : 0.987173 / 
04-#1436 : 0.997214 / 05-#0315 : 0.974603 

...


train loss 0.0223 acc 40532.0000 topk 40774.0000
valid loss (single) 0.0494 acc 0.9860 topk 0.9957
valid loss (others) 0.0652 acc 0.9887 topk 0.9971
lr 8.015160008714386e-05 time 18054.43s
-------------------- class (single) --------------------
00-#0228 : 0.995614 / 01-#0370 : 0.997297 / 02-#0008 : 0.500000 / 03-#0051 : 0.960784 / 
04-#0052 : 1.000000 / 05-#0007 : 0.571429 / 06-#0062 : 0.983871 / 07-#0185 : 0.983784 / 
08-#0011 : 1.000000 / 09-#0061 : 1.000000 / 10-#0002 : 1.000000 / 11-#0031 : 1.000000 / 
12-#0045 : 0.933333 / 13-#0492 : 0.997967 / 14-#0012 : 1.000000 / 15-#0017 : 1.000000 / 
16-#0104 : 0.980769 / 17-#0021 : 0.952381 / 18-#0036 : 0.944444 / 19-#0004 : 1.000000 / 
20-#0010 : 1.000000 / 21-#0027 : 1.000000 / 22-#0007 : 1.000000 / 23-#0005 : 0.800000 / 
24-#0002 : 1.000000 / 25-#0002 : 0.500000 / 
-------------------- class (others) --------------------
00-#0111 : 0.990991 / 01-#1109 : 0.993688 / 02-#0657 : 0.968037 / 03-#2105 : 0.994774 / 
04-#1436 : 0.998607 / 05-#03


train loss 0.0211 acc 40561.0000 topk 40778.0000
valid loss (single) 0.0426 acc 0.9897 topk 0.9973
valid loss (others) 0.0559 acc 0.9907 topk 0.9964
lr 7.23146308710381e-05 time 19813.92s
-------------------- class (single) --------------------
00-#0228 : 0.991228 / 01-#0370 : 0.997297 / 02-#0008 : 0.625000 / 03-#0051 : 0.960784 / 
04-#0052 : 1.000000 / 05-#0007 : 0.857143 / 06-#0062 : 0.983871 / 07-#0185 : 1.000000 / 
08-#0011 : 1.000000 / 09-#0061 : 1.000000 / 10-#0002 : 1.000000 / 11-#0031 : 1.000000 / 
12-#0045 : 0.977778 / 13-#0492 : 0.997967 / 14-#0012 : 0.833333 / 15-#0017 : 1.000000 / 
16-#0104 : 1.000000 / 17-#0021 : 0.952381 / 18-#0036 : 0.972222 / 19-#0004 : 1.000000 / 
20-#0010 : 1.000000 / 21-#0027 : 1.000000 / 22-#0007 : 0.857143 / 23-#0005 : 1.000000 / 
24-#0002 : 0.500000 / 25-#0002 : 0.500000 / 
-------------------- class (others) --------------------
00-#0111 : 0.981982 / 01-#1109 : 0.991885 / 02-#0657 : 0.989346 / 03-#2105 : 0.992399 / 
04-#1436 : 0.998607 / 05-#031


train loss 0.0139 acc 40645.0000 topk 40793.0000
valid loss (single) 0.0678 acc 0.9822 topk 0.9951
valid loss (others) 0.0617 acc 0.9887 topk 0.9966
lr 6.371946635410054e-05 time 21611.09s
-------------------- class (single) --------------------
00-#0228 : 0.978070 / 01-#0370 : 0.997297 / 02-#0008 : 0.500000 / 03-#0051 : 0.960784 / 
04-#0052 : 1.000000 / 05-#0007 : 0.714286 / 06-#0062 : 0.967742 / 07-#0185 : 0.994595 / 
08-#0011 : 1.000000 / 09-#0061 : 1.000000 / 10-#0002 : 1.000000 / 11-#0031 : 1.000000 / 
12-#0045 : 0.933333 / 13-#0492 : 0.995935 / 14-#0012 : 0.916667 / 15-#0017 : 1.000000 / 
16-#0104 : 0.990385 / 17-#0021 : 0.904762 / 18-#0036 : 0.944444 / 19-#0004 : 1.000000 / 
20-#0010 : 0.900000 / 21-#0027 : 1.000000 / 22-#0007 : 0.714286 / 23-#0005 : 0.800000 / 
24-#0002 : 1.000000 / 25-#0002 : 0.500000 / 
-------------------- class (others) --------------------
00-#0111 : 0.963964 / 01-#1109 : 0.988278 / 02-#0657 : 0.983257 / 03-#2105 : 0.996200 / 
04-#1436 : 0.999304 / 05-#03


train loss 0.0118 acc 40676.0000 topk 40803.0000
valid loss (single) 0.0649 acc 0.9806 topk 0.9946
valid loss (others) 0.0772 acc 0.9898 topk 0.9962
lr 5.465880455519194e-05 time 23431.51s
-------------------- class (single) --------------------
00-#0228 : 0.982456 / 01-#0370 : 0.989189 / 02-#0008 : 0.500000 / 03-#0051 : 0.941176 / 
04-#0052 : 1.000000 / 05-#0007 : 0.714286 / 06-#0062 : 0.983871 / 07-#0185 : 1.000000 / 
08-#0011 : 1.000000 / 09-#0061 : 1.000000 / 10-#0002 : 1.000000 / 11-#0031 : 1.000000 / 
12-#0045 : 0.933333 / 13-#0492 : 0.995935 / 14-#0012 : 1.000000 / 15-#0017 : 1.000000 / 
16-#0104 : 0.980769 / 17-#0021 : 0.904762 / 18-#0036 : 0.944444 / 19-#0004 : 0.750000 / 
20-#0010 : 0.800000 / 21-#0027 : 1.000000 / 22-#0007 : 0.857143 / 23-#0005 : 0.800000 / 
24-#0002 : 0.500000 / 25-#0002 : 0.500000 / 
-------------------- class (others) --------------------
00-#0111 : 0.963964 / 01-#1109 : 0.985573 / 02-#0657 : 0.996956 / 03-#2105 : 0.994299 / 
04-#1436 : 0.999304 / 05-#03


train loss 0.0087 acc 40713.0000 topk 40809.0000
valid loss (single) 0.0585 acc 0.9854 topk 0.9957
valid loss (others) 0.0640 acc 0.9907 topk 0.9974
lr 4.5441195444808064e-05 time 25209.71s
-------------------- class (single) --------------------
00-#0228 : 0.991228 / 01-#0370 : 1.000000 / 02-#0008 : 0.625000 / 03-#0051 : 0.980392 / 
04-#0052 : 1.000000 / 05-#0007 : 0.714286 / 06-#0062 : 0.983871 / 07-#0185 : 0.989189 / 
08-#0011 : 1.000000 / 09-#0061 : 1.000000 / 10-#0002 : 1.000000 / 11-#0031 : 1.000000 / 
12-#0045 : 0.955556 / 13-#0492 : 0.997967 / 14-#0012 : 1.000000 / 15-#0017 : 1.000000 / 
16-#0104 : 0.971154 / 17-#0021 : 0.904762 / 18-#0036 : 0.944444 / 19-#0004 : 0.750000 / 
20-#0010 : 0.900000 / 21-#0027 : 1.000000 / 22-#0007 : 0.857143 / 23-#0005 : 0.800000 / 
24-#0002 : 0.500000 / 25-#0002 : 0.500000 / 
-------------------- class (others) --------------------
00-#0111 : 0.954955 / 01-#1109 : 0.992786 / 02-#0657 : 0.998478 / 03-#2105 : 0.992874 / 
04-#1436 : 0.999304 / 05-#0

In [ ]:
valid_trues1==0